# NFLRadar Pro Football Reference Web Scraping Model
## Author: Basit Umair
## Description:
This script fetches NFL match data from Pro Football Reference using *BeautifulSoup* and modelled with *Pandas* into a dataframe.
Used as the data for the NFLRadar prediction module
### Inspired by DataQuest tutorial

In [4]:
import os
import time
from datetime import datetime
from io import StringIO

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [5]:
column_mapping = {  ## create a mapping of old names to clean names
    'unnamed:_0_level_0_week': 'week',
    'unnamed:_1_level_0_day': 'day',
    'unnamed:_2_level_0_date': 'date',
    'unnamed:_3_level_0_unnamed:_3_level_1': 'time',
    'unnamed:_4_level_0_unnamed:_4_level_1': 'boxscore_link',
    'unnamed:_5_level_0_unnamed:_5_level_1': 'result',
    'unnamed:_6_level_0_ot': 'overtime',
    'unnamed:_7_level_0_rec': 'record',
    'unnamed:_8_level_0_unnamed:_8_level_1': 'home_away',
    'unnamed:_9_level_0_opp': 'opponent',
    'score_tm': 'team_score',
    'score_opp': 'opponent_score',
    'offense_1std': 'offense_first_downs',
    'offense_totyd': 'offense_total_yards',
    'offense_passy': 'offense_passing_yards',
    'offense_rushy': 'offense_rushing_yards',
    'offense_to': 'offense_turnovers',
    'defense_1std': 'defense_first_downs_allowed',
    'defense_totyd': 'defense_total_yards_allowed',
    'defense_passy': 'defense_passing_yards_allowed',
    'defense_rushy': 'defense_rushing_yards_allowed',
    'defense_to': 'defense_turnovers_forced',
    'expected_points_offense': 'expected_points_offense',
    'expected_points_defense': 'expected_points_defense',
    'expected_points_sp._tms': 'expected_points_special_teams',
    'team': 'team',
    'season': 'season'
}
current_year = int(datetime.now().year)
years = list(range(current_year, current_year - 5, -1))  ## 2025, 2024, 2023, 2022, 2021

header = { ## be a respectful user
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        }

In [7]:
all_matches = []  ## list that will contain info for all matches
for year in years:  ## iterates from the current year down to 5 years ago. Ex. 2025 - 2021
    standings_url = f"https://www.pro-football-reference.com/years/{year}/index.htm"
    header = { ## be a respectful user
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }
    print(f"{standings_url}")

    data = requests.get(standings_url, headers=header)  ## getting HTML from the website
    soup = BeautifulSoup(data.text, features="html.parser")  ## parsing html
    afc_standings_table = soup.select('table.stats_table')[0]  ## get afc & nfc tables
    nfc_standings_table = soup.select('table.stats_table')[1]

    afc_links = afc_standings_table.find_all('a')  ## looking for anchors in the HTML to help find links
    nfc_links = nfc_standings_table.find_all('a')

    afc_links = [al.get("href") for al in afc_links]  ## get the hrefs from the anchors to get the urls
    afc_links = [al for al in afc_links if '/teams/' in al]  ## filter only for teams

    nfc_links = [nl.get("href") for nl in nfc_links]
    nfc_links = [nl for nl in nfc_links if '/teams/' in nl]

    links = afc_links + nfc_links  ## combine afc and nfc links
    teams_urls = [f"https://www.pro-football-reference.com{l}" for l in links]  ## specific links for each team
    time.sleep(8)  ## delay loop for 8 seconds to prevent being banned from scraping

    for team_url in teams_urls:  ## iterate through team urls
        team_abbreviation = team_url.split('/')[-2].upper()  ## get team abbreviations Ex. BUF
        try:
            team_data = requests.get(team_url, headers=header)
            matches = pd.read_html(StringIO(team_data.text), match="Schedule & Game Results")
            if matches:  # make sure we got some data
                team_schedule = matches[0]  # get the first DataFrame
                team_schedule['Team'] = team_abbreviation  # add metadata columns
                team_schedule['Season'] = year
                all_matches.append(team_schedule)  ## append to the main list
                print(f"{team_abbreviation} {year}")
            else:
                print(f"<———NO SCHEDULE DATA FOR {team_abbreviation}———>")
            time.sleep(8)  ## delay loop for 8 seconds to prevent being banned from scraping
        except ValueError as e:  ## if there's empty data for either the ongoing season or a Bye Week
            print(f"<———NO SCHEDULE DATA FOR {team_abbreviation}———>")
            continue

https://www.pro-football-reference.com/years/2025/index.htm
NWE 2025
BUF 2025
MIA 2025
NYJ 2025
PIT 2025
RAV 2025
CIN 2025
CLE 2025
CLT 2025
JAX 2025
HTX 2025
OTI 2025
DEN 2025
SDG 2025
KAN 2025
RAI 2025
PHI 2025
DAL 2025
WAS 2025
NYG 2025
DET 2025
CHI 2025
GNB 2025
MIN 2025
TAM 2025
CAR 2025
ATL 2025
NOR 2025
RAM 2025
SEA 2025
SFO 2025
CRD 2025
https://www.pro-football-reference.com/years/2024/index.htm
BUF 2024
MIA 2024
NYJ 2024
NWE 2024
RAV 2024
PIT 2024
CIN 2024
CLE 2024
HTX 2024
CLT 2024
JAX 2024
OTI 2024
KAN 2024
SDG 2024
DEN 2024
RAI 2024
PHI 2024
WAS 2024
DAL 2024
NYG 2024
DET 2024
MIN 2024
GNB 2024
CHI 2024
TAM 2024
ATL 2024
CAR 2024
NOR 2024
RAM 2024
SEA 2024
CRD 2024
SFO 2024
https://www.pro-football-reference.com/years/2023/index.htm
BUF 2023
MIA 2023
NYJ 2023
NWE 2023
RAV 2023
CLE 2023
PIT 2023
CIN 2023
HTX 2023
JAX 2023
CLT 2023
OTI 2023
KAN 2023
RAI 2023
DEN 2023
SDG 2023
DAL 2023
PHI 2023
NYG 2023
WAS 2023
DET 2023
GNB 2023
MIN 2023
CHI 2023
TAM 2023
NOR 2023
ATL 2023
C

In [8]:
if all_matches:  ## concatenate the DataFrames
    match_df = pd.concat(all_matches, ignore_index=True)  ## format into a dataframe
    if isinstance(match_df.columns, pd.MultiIndex):
        match_df.columns = ['_'.join(col).strip('_') for col in match_df.columns.values]  # flatten MultiIndex columns by joining the levels
    else:
        match_df.columns = match_df.columns.astype(str)  ## convert regular columns to strings
    match_df.columns = [col.lower().replace(' ', '_').replace('(', '').replace(')', '') for col in match_df.columns]  ## formating columns
    match_df = match_df.rename(columns=column_mapping)  ## apply the mapping

    os.makedirs("data", exist_ok=True)
    match_df.to_csv("data/2021_2025_matches.csv", index=False)  ## export to csv file without index column

In [9]:
match_df.head()

,week,day,date,time,boxscore_link,result,overtime,record,home_away,opponent,...,defense_first_downs_allowed,defense_total_yards_allowed,defense_passing_yards_allowed,defense_rushing_yards_allowed,defense_turnovers_forced,expected_points_offense,expected_points_defense,expected_points_special_teams,team,season
0,1,Sun,September 7,1:00PM ET,boxscore,L,NaN,0-1,NaN,Las Vegas Raiders,...,21.0,389.0,333.0,56.0,1.0,1.07,-7.21,-4.37,NWE,2025
1,2,Sun,September 14,1:00PM ET,boxscore,W,NaN,1-1,@,Miami Dolphins,...,16.0,358.0,297.0,61.0,1.0,12.82,-5.66,-3.11,NWE,2025
2,3,Sun,September 21,1:00PM ET,boxscore,L,NaN,1-2,NaN,Pittsburgh Steelers,...,17.0,203.0,139.0,64.0,1.0,-2.18,-2.22,-1.65,NWE,2025
3,4,Sun,September 28,1:00PM ET,boxscore,W,NaN,2-2,NaN,Carolina Panthers,...,21.0,326.0,197.0,129.0,NaN,14.44,-0.46,12.06,NWE,2025
4,5,Sun,October 5,8:20PM ET,boxscore,W,NaN,3-2,@,Buffalo Bills,...,25.0,363.0,245.0,118.0,3.0,8.03,-6.83,0.8,NWE,2025
